In [1]:
import pandas as pd
import numpy as np

In [22]:
df_orders = pd.read_csv('orders.csv')
df_orders['order_month'] = df_orders['order_date'].apply(lambda x: x[0:7])
df_orders['order_year'] = df_orders['order_date'].apply(lambda x: x[0:4])

In [23]:
df_customers = pd.read_csv('customers.csv')
cols = ['customer_id']
cols.extend(df_customers.columns[1:])
df_customers.columns = cols

In [24]:
df_orders.sample(5)

,id,order_date,ship_mode,customer_id,sales,order_month,order_year
3946,157651,2017-12-10,Standard,HA-14920,14.200,2017-12,2017
4716,169040,2016-12-06,Standard,GT-14710,1169.454,2016-12,2016
1811,107615,2016-03-22,First,RB-19645,58.580,2016-03,2016
767,166730,2014-12-30,First,DD-13570,39.128,2014-12,2014
1778,105473,2016-04-16,Second,BM-11785,28.800,2016-04,2016


In [25]:
df_customers.sample(5)

,customer_id,name,segment,state,city
396,NC-18340,Nat Carroll,Consumer,Michigan,Lansing
652,MS-17530,MaryBeth Skach,Consumer,California,San Diego
596,MW-18220,Mitch Webber,Consumer,New York,New York City
356,TZ-21580,Tracy Zic,Consumer,Colorado,Louisville
20,ES-14080,Erin Smith,Corporate,Florida,Melbourne


In [26]:
df_merge = pd.merge(df_orders, df_customers, 'inner', on='customer_id')

In [27]:
df_merge.sample(5)

,id,order_date,ship_mode,customer_id,sales,order_month,order_year,name,segment,state,city
3631,126445,2015-08-27,Standard,RA-19945,484.650,2015-08,2015,Ryan Akin,Consumer,California,Murrieta
4544,158701,2015-01-05,Standard,JL-15175,143.520,2015-01,2015,James Lanier,Home Office,Tennessee,Columbia
3363,116722,2016-11-11,Standard,LP-17080,524.616,2016-11,2016,Liz Pelletier,Consumer,California,San Francisco
431,131597,2015-09-14,Standard,SP-20620,170.136,2015-09,2015,Stefania Perrino,Corporate,Pennsylvania,Philadelphia
917,113327,2015-05-26,Second,BF-11170,352.168,2015-05,2015,Ben Ferrer,Home Office,California,Los Angeles


In [28]:
df_first = df_merge.groupby('customer_id')['order_year'].min().reset_index()
df_first.columns = ['customer_id', 'first_order']

In [29]:
df_first.sample(5)

,customer_id,first_order
117,BW-11065,2014
190,DB-13060,2014
258,EH-13945,2014
789,YC-21895,2014
132,CC-12430,2015


In [30]:
df_full = pd.merge(df_merge, df_first, 'inner', on='customer_id')

In [31]:
df_full.sample(5)

,id,order_date,ship_mode,customer_id,sales,order_month,order_year,name,segment,state,city,first_order
1706,116981,2015-03-26,Second,SM-20950,1098.530,2015-03,2015,Suzanne McNair,Corporate,North Carolina,Greenville,2014
3240,133004,2017-08-31,Standard,AJ-10945,638.730,2017-08,2017,Ashley Jarboe,Consumer,North Carolina,Wilmington,2014
3284,144694,2017-09-24,Second,BD-11605,253.824,2017-09,2017,Brian Dahlen,Consumer,Massachusetts,Lawrence,2014
736,108056,2016-09-29,Standard,ES-14080,859.200,2016-09,2016,Erin Smith,Corporate,Florida,Melbourne,2014
4455,118570,2016-11-13,Standard,CC-12430,217.056,2016-11,2016,Chuck Clark,Home Office,Indiana,Columbus,2015


In [46]:
df_users = df_full.groupby(['customer_id', 'first_order', 'order_year'])\
    .agg({'sales':'sum', 'id':'count'}).reset_index()

In [47]:
df_users.sample(5)

,customer_id,first_order,order_year,sales,id
516,CR-12580,2016,2016,275.918,3
742,DV-13045,2016,2016,37.600,2
762,Dl-13600,2014,2017,16.696,2
1300,KB-16240,2014,2014,1.869,1
676,DL-12865,2014,2015,19.650,1


In [40]:
def get_crosstab(df, value_column):
    df_crosstab = pd.crosstab(df.customer_id, df.order_year, values=df[value_column], aggfunc=sum)\
        .reset_index()
    df_crosstab = pd.merge(df_crosstab, df_first, how='inner', on='customer_id')
    del df_crosstab['customer_id']
    return df_crosstab

In [53]:
df_test = get_crosstab(df_users, 'id')

In [54]:
df_test.head(5)

,2014,2015,2016,2017,first_order
0,2.0,1.0,1.0,1.0,2014
1,2.0,3.0,2.0,2.0,2014
2,1.0,NaN,2.0,1.0,2014
3,2.0,1.0,2.0,1.0,2014
4,2.0,NaN,1.0,NaN,2014


In [41]:
def get_retention(df):
    df_triangle = df.groupby(['first_order']).sum().reset_index()
    df_triangle['Base'] = np.diag(df_triangle.iloc[:,1:])
    df_retention = pd.DataFrame(df_triangle[['first_order', 'Base']])
    df_retention.columns = ['Cohort', 'Base']

    for col in df_triangle.columns[1:-1]:
        df_retention[col] = round(df_triangle[col] / df_triangle['Base'], 3)

    for i in range(2, len(df_retention.columns)):
        df_retention.iloc[i-1:, i] = None
        
    return df_retention

### Sales retention

In [50]:
get_retention(get_crosstab(df_users, 'id'))

,Cohort,Base,2014,2015,2016,2017
0,2014,969.0,1.0,0.833,1.028,1.288
1,2015,231.0,NaN,1.000,0.931,1.268
2,2016,104.0,NaN,NaN,1.000,1.221
3,2017,19.0,NaN,NaN,NaN,1.000


### Transaction retention

In [17]:
get_retention(get_crosstab(df_users, 'id'))

,Cohort,Base,2014,2015,2016,2017
0,2014,969.0,1.0,0.833,1.028,1.288
1,2015,231.0,NaN,1.000,0.931,1.268
2,2016,104.0,NaN,NaN,1.000,1.221
3,2017,19.0,NaN,NaN,NaN,1.000


### Sales retention by segment

In [18]:
for segment in df_users.segment.unique():
    print(f'\nSegment: {segment}')
    print(get_retention(get_crosstab(df_users[df_users.segment==segment], 'sales')))


Segment: Consumer
  Cohort         Base  2014   2015   2016   2017
0   2014  266096.8126   1.0  0.789  0.818  0.906
1   2015   56590.8551   NaN  1.000  0.832  0.817
2   2016   32119.4550   NaN    NaN  1.000  1.229
3   2017    4985.0760   NaN    NaN    NaN  1.000

Segment: Home Office
  Cohort        Base  2014   2015   2016   2017
0   2014  89715.8118   1.0  0.616  0.874  1.296
1   2015  19942.1216   NaN  1.000  1.120  1.475
2   2016   4483.5420   NaN    NaN  1.000  2.905
3   2017    729.6480   NaN    NaN    NaN  1.000

Segment: Corporate
  Cohort         Base  2014   2015   2016   2017
0   2014  128434.8737   1.0  0.778  1.237  1.367
1   2015   28855.5195   NaN  1.000  1.043  1.944
2   2016   18200.6230   NaN    NaN  1.000  0.462
3   2017    1797.0810   NaN    NaN    NaN  1.000


### Sales retention by ship_mode

In [19]:
for ship_mode in df_users.ship_mode.unique():
    print(f'\nShip_mode: {ship_mode}')
    print(get_retention(get_crosstab(df_users[df_users.ship_mode==ship_mode], 'sales')))


Ship_mode: Standard
  Cohort         Base  2014   2015   2016   2017
0   2014  305621.3196   1.0  0.716  0.921  0.935
1   2015   65723.8550   NaN  1.000  0.791  1.142
2   2016   38855.4240   NaN    NaN  1.000  0.752
3   2017    5644.8960   NaN    NaN    NaN  1.000

Ship_mode: First
  Cohort        Base  2014   2015   2016   2017
0   2014  59769.2645   1.0  0.997  1.010  1.590
1   2015   9644.0606   NaN  1.000  1.632  2.143
2   2016   6153.2850   NaN    NaN  1.000  3.814
3   2017    955.7840   NaN    NaN    NaN  1.000

Ship_mode: Second
  Cohort         Base  2014   2015   2016   2017
0   2014  101386.7800   1.0  0.642  0.811  1.105
1   2015   23970.2056   NaN  1.000  1.180  1.216
2   2016    9537.8710   NaN    NaN  1.000  0.697
3   2017     911.1250   NaN    NaN    NaN  1.000

Ship_mode: Same Day
  Cohort       Base  2014   2015   2016   2017
0   2014  17470.134   1.0  1.234  1.759  2.308
1   2015   6050.375   NaN  1.000  0.581  1.136
2   2016    257.040   NaN    NaN  1.000  6.163


### Transaction retention by segment

In [20]:
for segment in df_users.segment.unique():
    print(f'\nSegment: {segment}')
    print(get_retention(get_crosstab(df_users[df_users.segment==segment], 'id')))


Segment: Consumer
  Cohort   Base  2014   2015   2016   2017
0   2014  524.0   1.0  0.817  0.954  1.265
1   2015  107.0   NaN  1.000  0.869  1.215
2   2016   58.0   NaN    NaN  1.000  1.207
3   2017   13.0   NaN    NaN    NaN  1.000

Segment: Home Office
  Cohort   Base  2014   2015   2016   2017
0   2014  158.0   1.0  0.899  1.095  1.405
1   2015   49.0   NaN  1.000  1.122  1.429
2   2016   14.0   NaN    NaN  1.000  1.786
3   2017    1.0   NaN    NaN    NaN  1.000

Segment: Corporate
  Cohort   Base  2014   2015   2016   2017
0   2014  287.0   1.0  0.826  1.125  1.265
1   2015   75.0   NaN  1.000  0.893  1.240
2   2016   32.0   NaN    NaN  1.000  1.000
3   2017    5.0   NaN    NaN    NaN  1.000


### Transaction retention by ship_mode

In [21]:
for ship_mode in df_users.ship_mode.unique():
    print(f'\nShip_mode: {ship_mode}')
    print(get_retention(get_crosstab(df_users[df_users.ship_mode==ship_mode], 'id')))


Ship_mode: Standard
  Cohort   Base  2014   2015   2016   2017
0   2014  586.0   1.0  0.845  1.002  1.229
1   2015  141.0   NaN  1.000  0.943  1.284
2   2016   62.0   NaN    NaN  1.000  1.242
3   2017   12.0   NaN    NaN    NaN  1.000

Ship_mode: First
  Cohort   Base  2014   2015   2016   2017
0   2014  145.0   1.0  0.772  1.103  1.448
1   2015   31.0   NaN  1.000  1.194  1.516
2   2016   18.0   NaN    NaN  1.000  1.278
3   2017    4.0   NaN    NaN    NaN  1.000

Ship_mode: Second
  Cohort   Base  2014   2015   2016   2017
0   2014  190.0   1.0  0.826  0.995  1.311
1   2015   49.0   NaN  1.000  0.673  1.020
2   2016   22.0   NaN    NaN  1.000  1.000
3   2017    3.0   NaN    NaN    NaN  1.000

Ship_mode: Same Day
  Cohort  Base  2014   2015  2016   2017
0   2014  48.0   1.0  0.896  1.25  1.438
1   2015  10.0   NaN  1.000  1.20  1.500
2   2016   2.0   NaN    NaN  1.00  2.500


## Ответ
#### 1. Тотально sales и transaction ретеншены 2016 лучше 2015
#### 2. Но есть слабые когорты, где хуже: 
#### - Для sales-retention это:
    - Segment: Consumer, first order=2015
    - Ship_mode: Standard, first order=2015
#### - Для transaction-retention это:
    - Segment: Consumer, first order=2015
    - Segment: Corporate, first order=2015
    - Ship_mode: Standard, first order=2015
    - Ship_mode: Second, first order=2015
